## Qual porcentagem da variação dos dados é explicada pelos supostos previsores em uma regressão linear?

A resposta é o R2 Score no caso de uma regressão com apenas 1 preditor e o R2 ajustado no caso de uma regressão com 2 ou mais preditores.

R2 = SSR / SST

R2 Ajustado = ( SSR / (nAmostras / nPreditores) ) / (SST / (nAmostras - 1))

O R2 ajustado é sempre menor ou igual ao R2 normal

In [3]:
SST = 850.6
SSR = 670.3

R2 = (SSR / SST)
print(R2)

0.788031977427698


## Teste F - Quais preditores explicam uma fração signigicativa da variação?

Saber se os previsores explicam uma fração significativa da variação equivale a perguntar se a regressao é significativa. 

Quem responde se a regressão é significativa é o teste-F.

Teste F:

SST = 850.6

SSR = 670.3


SSE = SST - SSR = 850.6 - 670.3 = 180.3


Graus de Liberdade do SSR é: número de previsores (o B0 não é um previsor!)

Graus de Liberdade do SST é: número de amostras - 1

Graus de liberdade do SSE é: número de amostras - número de previsores - 1


MSR = SSR / Graus de Liberdade do SSR = 670.3 / 2 = 335.15

MSE = SSE / Graus de Liberdade do SSE = 180.3 / 25 = 7.212

F Calculado = MSR / MSE = 46.7

In [1]:
SST = 850.6
SSR = 670.3
SSE = SST - SSR

dofSSR = 2
dofSST = 28 - 1
dofSSE = 28 - 2 - 1

MSR = SSR / dofSSR
MSE = SSE / dofSSE
print(f"MSR = {MSR:.4f}, MSE = {MSE:.4f}")

calculatedF = MSR / MSE
print(f"Calculated F = {calculatedF}")

MSR = 335.1500, MSE = 7.2120
Calculated F = 46.47115917914585


Agora, pegamos o F calculado e comparamos com o valor na tabela do teste F.

https://statisticsbyjim.com/hypothesis-testing/f-table/

No nosso exemplo, temos o dofSSR = 2, o dofSSE = 25 e queremos 90% de confiança. Assim, vamos na tabela do 90% de confiança e pegamos o valor tabela[2][25] = 2.53
Com 95% de confiança, iríamos na tabela do 95% e pegamos o mesmo valor tabela[2][25] = 3.39

Isso tudo para, no final, sabermos que a nossa regressão passa no teste F porque o nosso F calculado é maior que os valores encontrados na tabela F. 

É literalmente só fazer: F calculado > valor da tabela F? Se sim, passou. Se não, não passou.

## Preditores significativos


Para saber quais previsores são significativos, fazemos um intervalo de confiança para cada um dos previsores :)

https://www.ttable.org/

Continuando no nosso exemplo, temos que:

dof = nAmostras - nPreditores - 1 = 28 - 2 - 1 = 25

Então, vamos na tabela e buscamos o valor na coluna two-tails. Para 95% de confiança e 25 graus de liberdade, no two-tails temos que o valor buscado é 2.06

In [2]:
valorTabelaT = 1.740

# Como o coeficiente 0 não é um preditor, não incluímos ele!
coeficiente1 = 0.02
coeficiente2 = 3

desviopadraocoeficiente1 = 0.0015
desviopadraocoeficiente2 = 0.9

for idx, (c, s) in enumerate(zip([coeficiente1, coeficiente2], [desviopadraocoeficiente1, desviopadraocoeficiente2])):
    lowerBound = c - s * valorTabelaT
    upperBound = c + s * valorTabelaT

    print(f"Coeficiente {idx+1}: [{lowerBound:.4f}, {upperBound:.4f}]")

Coeficiente 1: [0.0174, 0.0226]
Coeficiente 2: [1.4340, 4.5660]


## Há algum indício de multicolinearidade entre A e B? Por quê? Em caso afirmativo, descreva o que deverá ser feito para tratar este problema.

Não há indício algum, pois o valor do R2 está bom, o teste F passou e o IC dos dois preditores é significativo. Não precisamos fazer mais nada :)

## Vendo se um modelo com apenas um previsor compensa:

In [3]:
import numpy as np

# Calculando os previsores (no caso aqui é só 1 porque o b0 não é um previsor, mas enfim)
# Ainda precisamos do b0 para fazer a regressão, então bora.

valoresX = [8.5, 8.9, 10.6, 11.6, 13, 13.2]
valoresY = [30.9, 32.7, 36.7, 46.3, 46.2, 47.8]

mediaX = np.mean(valoresX)
mediaY = np.mean(valoresY)

print(f"MediaX = {mediaX:.4f}, MediaY = {mediaY:.4f}")

sigmaXY = 0
sigmaX2 = 0
for x, y in zip(valoresX, valoresY):
    sigmaXY += x * y
    sigmaX2 += x * x

print(f"SigmaXY = {sigmaXY:.4f}, SigmaX^2 = {sigmaX2:.4f}")

previsor1 = (sigmaXY - len(valoresX) * mediaX * mediaY) / (sigmaX2 - len(valoresX) * mediaX * mediaX)
previsor0 = mediaY - previsor1 * mediaX

print(f"Coeficiente Previsor 0 = {previsor0:.4f}, Coeficiente Previsor 1 = {previsor1:.4f}")

# Agora que temos o modelo da regressão com os previsores, calculados os SSR e o SST para
# calcular o R2 Score. Quem vai dizer se a regressão está boa é o R2.

# O SSR é o SST - SSE, então vamos começar calculando o SSE.

# O SSE é a soma dos erros ao quadrado. Vamos lá:

modelo = lambda x: previsor0 + previsor1 * x
SSE    = 0

for x, y in zip(valoresX, valoresY):
    predictedY   = modelo(x)
    errorSquared = (predictedY - y) ** 2
    SSE += errorSquared

print(f"SSE = {SSE:.4f}")

# Agora, calcular o SSY.
# O SSY é a soma de todos os Y ao quadrado. Vamos lá:
SSY = 0
for y in valoresY:
    SSY += y ** 2

print(f"SSY = {SSY:.4f}")

# Por fim, calculamos o SST. 
# O SST é a soma dos quadrados das diferenças entre a média do Y e o Y real. É simples, eu juro. Olha abaixo:
SST = 0
for y in valoresY:
    SST += (y - mediaY) ** 2

print(f"SST = {SST:.4f}")

# Já temos todos os componentes para fazer o cálculo do R2! Bora que tá quase!
# O SSR é o SST - SSE, e já temos os dois!

SSR = SST - SSE
print(f"SSR = {SSR:.4f}")

# Por fim, calculamos o R2:
R2 = SSR / SST
print(f"R2 = {R2:.4f}")

MediaX = 10.9667, MediaY = 40.1000
SigmaXY = 2711.3400, SigmaX^2 = 741.6200
Coeficiente Previsor 0 = 0.2298, Coeficiente Previsor 1 = 3.6356
SSE = 21.3755
SSY = 9933.9600
SST = 285.9000
SSR = 264.5245
R2 = 0.9252


## Exemplo 2 de achar uma regressão para dados e ver se compensa

Tenho os dados a seguir:

| K    | Tempo (ms) |
|------|------------|
| 128  | 93         |
| 256  | 478        |
| 512  | 3404       |
| 1024 | 25410      |

E aí? Uma regressão linear pode ser aplicada aos dados diretamente? Não, porque eles não seguem um padrão linear... Quando o K dobra, o tempo mais do que dobra, e muito mais. Quando o K dobra de novo, o tempo de novo aumenta ainda mais.

Aqui esses dados crescem de maneira exponencial. O que significa que o nosso modelo, a princípio, é exponencial...

Calma, isso só significa que a fórmula dele deixa de ser y = a*x + b e vira
y = a * e^(bx)

Mas... e se a gente tirar um log na base 'e' desse modelo? Fica:

ln(y) = ln(a) + bx * ln(e) // vira '+' porque ln(a * b) = ln(a) + ln(b)! :)

ln(y) = ln(a) + bx * 1

ln(y) = ln(a) + bx

Olha só, virou um modelo linear! Vamos trocar esse ln(a) por outro nome: b0. Também vamos trocar ln(y) por y'.

Fica: 

y' = b0 + b1 * x

Virou um modelo linear que agora podemos reciclar as mesmas contas de antes!

In [4]:
valoresX = [128, 256, 512, 1024]
valoresY = [93, 478, 3404, 25410]

# Não podemos mexer nos valores de X. Isso porque, no nosso modelo, o 'x' da fórmula exponencial 
# nunca ficou com um ln. Quem ficou com ln foi o y, no log(y). Por isso, tiramos log apenas dos valores Y.
valoresXLog = valoresX
valoresYLog = np.log(valoresY)

mediaX = np.mean(valoresXLog)
mediaY = np.mean(valoresYLog)

print(f"MediaX = {mediaX:.4f}\nMediaY = {mediaY:.4f}")

sigmaXY = 0
sigmaX2 = 0
for x, y in zip(valoresXLog, valoresYLog):
    sigmaXY += x * y
    sigmaX2 += x * x

print(f"SigmaXY = {sigmaXY:.4f}\nSigmaX^2 = {sigmaX2:.4f}")

previsor1 = (sigmaXY - len(valoresX) * mediaX * mediaY) / (sigmaX2 - len(valoresX) * mediaX * mediaX)
previsor0 = mediaY - previsor1 * mediaX

print(f"Coeficiente Previsor 0 = {previsor0:.4f}")
print(f"Coeficiente Previsor 1 = {previsor1:.4f}")

# Agora que temos o modelo da regressão com os previsores, calculados os SSR e o SST para
# calcular o R2 Score. Quem vai dizer se a regressão está boa é o R2.

# O SSR é o SST - SSE, então vamos começar calculando o SSE.

# O SSE é a soma dos erros ao quadrado. Vamos lá:

modelo = lambda x: previsor0 + previsor1 * x
SSE    = 0

for x, y in zip(valoresXLog, valoresYLog):
    predictedY   = modelo(x)
    errorSquared = (predictedY - y) ** 2
    SSE += errorSquared

print(f"SSE = {SSE:.4f}")

# Agora, calcular o SSY.
# O SSY é a soma de todos os Y ao quadrado. Vamos lá:
SSY = 0
for y in valoresYLog:
    SSY += y ** 2

print(f"SSY = {SSY:.4f}")

# Por fim, calculamos o SST. 
# O SST é a soma dos quadrados das diferenças entre a média do Y e o Y real. É simples, eu juro. Olha abaixo:
SST = 0
for y in valoresYLog:
    SST += (y - mediaY) ** 2

print(f"SST = {SST:.4f}")

# Já temos todos os componentes para fazer o cálculo do R2! Bora que tá quase!
# O SSR é o SST - SSE, e já temos os dois!

SSR = SST - SSE
print(f"SSR = {SSR:.4f}")

# Por fim, calculamos o R2:
R2 = SSR / SST
print(f"R2 = {R2:.4f}")

MediaX = 480.0000
MediaY = 7.2445
SigmaXY = 16709.8664
SigmaX^2 = 1392640.0000
Coeficiente Previsor 0 = 4.3907
Coeficiente Previsor 1 = 0.0059
SSE = 1.0493
SSY = 227.6279
SST = 17.6994
SSR = 16.6501
R2 = 0.9407


## Quais parâmetros são significativos?

Já temos os valores dos coeficientes: 4.3907 e 0.0059.

O valor dos graus de liberdade é: 

dof = Número de amostras - nPreditores - 1

dof = 4 - 1 - 1 

dof = 2

Calcular o desvio padrão dos coeficientes é mais chatinho, vamos fazer um programa em Python abaixo:

In [4]:
import numpy as np

valoresX = [128, 256, 512, 1024]
valoresY = [93, 478, 3404, 25410]

mediaX = 480.0000
mediaY = 7.2445

sigmaXY = 16709.8664
sigmaX2 = 1392640.0000

Coeficiente0 = 4.3907
Coeficiente1 = 0.0059

SSE = 1.0493
SSY = 227.6279
SST = 17.6994
SSR = 16.6501
R2 = 0.9407

Se = np.sqrt(SSE / (len(valoresX) - 2))
#Se = round(Se, 4)
print(f"Se = {Se:.4f}")

Sb0 = Se * np.sqrt((1 / len(valoresX)) + ((mediaX ** 2) / (sigmaX2 - len(valoresX) * mediaX ** 2)))
print(f"Sb0 = {Sb0:.4f}")

Sb1 = Se / (np.sqrt(sigmaX2 - len(valoresX) * mediaX ** 2))
print(f"Sb1 = {Sb1:.4f}")

# Pronto! Agora, tiramos os ICs da mesma forma que fizemos antes!

# IC de 90% two-tails na tabela T para 2 graus de liberdade é 2.92!
# https://www.ttable.org/

valorTabelaT = 2.92
for idx, (c, s) in enumerate(zip([Coeficiente0, Coeficiente1], [Sb0, Sb1])):
    lowerBound = c - s * valorTabelaT
    upperBound = c + s * valorTabelaT

    print(f"IC para Coeficiente {idx+1}: [{lowerBound:.4f}, {upperBound:.4f}]")

Se = 0.7243
Sb0 = 0.6227
Sb1 = 0.0011
IC para Coeficiente 1: [2.5723, 6.2091]
IC para Coeficiente 2: [0.0028, 0.0090]


## Qual o tempo esperado para criptografar um registro de 384 bits?

Sem medo! Joga na fórmula!

y' = b0 + b1 * x

y' = 4.3907 + 0.005945 * 384

In [80]:
yLinha = 4.3909 + 0.005945 * 384

# Temos que lembrar que esse yLinha é o ln do Y real. Para obter o Y real, é só desfazer o ln.
# Elevamos 'e' ao valor yLinha para obter o Y real.

yReal = np.e ** yLinha
print(f"Y = {yReal:.4f}")

Y = 791.3814


## Quais limites  você colocaria para esta estimativa se você  aceita um erro máximo de 10% para uma única medida futura?

Esse eu não entendi como faz nem lendo a correção dela :(

## A regressão múltipla é significativa com 90% de confiança?

Então, é só fazer o teste F :)

Quem diz pra gente se a regressão é significativa é o teste F.

Nesse tipo de questão, ela TEM que dar as matrizes resultantes.

Aqui, temos os seguintes dados:

| j   | 0       | 1      | 2      |
|-----|---------|--------|--------|
| Bj  | -0.1614 | 0.1182 | 0.0165 |
| Cjj | 0.6297  | 0.0280 | 0.0012 |

^^^^
O valor na linha B é o valor dos coeficientes.
O valor na linha C é para calcular o desvio padrão de cada coeficiente.

Coeficiente de correlação múltipla é apenas outro nome para o R. Atenção, não é o R², é o R mesmo! 

R = 0.99

O desvio padrão dos erros é 1.2
Se = 1.2

In [7]:
# Com essas informações, já conseguimos fazer um bocado de coisas:

R  = 0.99
R2 = R * R
Se = 1.2

nAmostras   = 7
nParametros = 2

# Se = np.sqrt(SSE / (nAmostras - nParametros - 1))
# Se ^2 = SSE / (nAmostras - nParametros - 1)
# Se ^2 = SSE / (7 - 2 - 1)
# SSE = Se ^ 2 * 4
SSE = Se ** 2 * 4

MSE = Se ** 2 

# SST = SSR + SSE
# R2 = SSR / SST
# R2 = SSR / (SSR + SSE)
# R2 * SSR + R2 * SSE = SSR
# SSR = 0.9801 * SSR + 5.6454
# 0.0199 * SSR = 5.6454
SSR = 283.6884

MSR = SSR / nParametros

# Calculando o valor F...
calculatedF = MSR / MSE
print(f"Valor F = {calculatedF:.4f}")

# Agora, vamos na tabela F e pegamos o valor tabela[dofSSR][dofSSE] :) 
dofSSR = nParametros
dofSSE = nAmostras - nParametros - 1

print(f"Graus de Liberdade SSR = {dofSSR}")
print(f"Graus de Liberdade SSE = {dofSSE}")

# https://statisticsbyjim.com/hypothesis-testing/f-table/
valorTabelaF = 4.32

if calculatedF > valorTabelaF:
    print(f"A regressão é significativa pois o F Calculado {calculatedF:.4f} é maior que o valor na tabela F {valorTabelaF:.4f}")
else:
    print(f"A regressão NÃO é significativa pois o F Calculado {calculatedF:.4f} é menor que o valor na tabela F {valorTabelaF:.4f}")

Valor F = 98.5029
Graus de Liberdade SSR = 2
Graus de Liberdade SSE = 4
A regressão é significativa pois o F Calculado 98.5029 é maior que o valor na tabela F 4.3200


## Quais variáveis previsoras são significativas com 90% de confiança?

Aqui temos que tirar o IC dos previsores.

B0 não é previsor! Então, não precisamos calcular ele!


Graus de Liberdade para os preditores = nAmostras - nPreditores - 1 = 7 - 2 -1 = 4

In [15]:
import numpy as np

nAmostras   = 7
nParametros = 2

R  = 0.99
R2 = R * R
Se = 1.2


# Sb1 = Se * np.sqrt(C[1][1])
Sb1 = Se * np.sqrt(0.0280)
print(f"Sb1 = {Sb1:.4f}")

# Sb2 = Se * np.sqrt(C[2][2])
Sb2 = Se * np.sqrt(0.0012)
print(f"Sb2 = {Sb2:.4f}")

# Agora, vamos na tabela T olhar o valor two-tailed para 90% de confiança.
# https://www.ttable.org/

# dof = nAmostras - nPreditores - 1 = 7 - 2 -1 = 4
dof = 4

# O valor na tabela T é o valor two-tailed para 90% de confiança com 4 graus de liberdade
valorTabelaT = 2.132

coeficiente1 = 0.1182
coeficiente2 = 0.0165

for idx, (c, s) in enumerate(zip([coeficiente1, coeficiente2], [Sb1, Sb2])):
    lowerBound = c - valorTabelaT * s
    upperBound = c + valorTabelaT * s

    print(f"Coeficiente {idx+1} tem IC: [{lowerBound:.4f}, {upperBound:.4f}]")

Sb1 = 0.2008
Sb2 = 0.0416
Coeficiente 0 tem IC: [-0.3099, 0.5463]
Coeficiente 1 tem IC: [-0.0721, 0.1051]


## Qual variável previsora você consegue estimar com maior precisão, para uma confiança de 90%? Justifique apresentando a precisão que você pode atribuir a cada variável com a dada confiança

Essa conta é um pouco confusa porque ela depende dos valores obtidos anteriormente no cálculo do IC.

O bom é que ela recicla esses mesmos valores, então não temos que recalcular mais nada.

In [20]:
for idx, (c, s) in enumerate(zip([coeficiente1, coeficiente2], [Sb1, Sb2])):
    erroMax = s * valorTabelaT / c

    print(f"Erro máximo associado à estimativa do preditor {idx+1}: {erroMax:.4f}")

Erro máximo associado à estimativa do preditor 1: 3.6218
Erro máximo associado à estimativa do preditor 2: 5.3712


Erro máximo associado à estimativa do preditor 1: 3.6218
Erro máximo associado à estimativa do preditor 2: 5.3712

Com os resultados acima, aparentemente a maior precisão é da variável B1, porque o erro dela é menor. Mas, esta precisão está muito baixa.

## Você está satisfeito com seu modelo? Justifique. Se não, qual seria seu próximo passo?

Então... O modelo tem um R2 bom, o teste F passou, mas nenhum preditor é significativo porque o IC dos dois inclui o 0. E a precisão das variáveis ficou ruim...

Isso é um sinal de multicolinearidade. O correto a se fazer nesse caso é criar testes separados com cada previsor e ver se o resultado melhora.

In [ ]:
R2 = 840.5 / (840.5+208.25)
print(R2)

0.8014302741358761


#### Exemplo de regressão multivariada

Essa é a tabela

| Esquema | #Ataques | Duração | Índice |
|---------|----------|---------|--------|
| A       | 5        | 118     | 8.1    |
| B       | 13       | 132     | 6.8    |
| C       | 20       | 119     | 7.0    |
| D       | 28       | 153     | 7.4    |
| E       | 41       | 91      | 7.7    |
| F       | 49       | 118     | 7.5    |
| G       | 61       | 132     | 7.6    |
| H       | 62       | 105     | 8.0    |

In [32]:
import numpy as np
''' Exemplo da lista
X = np.matrix([
                [1,  5, 118],
                [1, 13, 132],
                [1, 20, 119],
                [1, 28, 153],
                [1, 41, 91],
                [1, 49, 118],
                [1, 61, 132],
                [1, 62, 105]
]
)

Y = np.matrix([
        [8.1],
        [6.8],
        [7.0],
        [7.4],
        [7.7],
        [7.5],
        [7.6],
        [8.0]
])
'''

X = np.matrix([
                [1,  6, 16],
                [1,  6, 16],
                [1,  6, 16],
                [1,  3, 16],
                [1,  3, 16],
                [1,  3, 16],
                [1,  6, 8],
                [1,  6, 8],
                [1,  6, 8],
                [1,  3, 8],
                [1,  3, 8],
                [1,  3, 8]
]
)

Y = np.matrix([
        [29.97],
        [30.63],
        [29.90],
        [30.03],
        [29.12],
        [30.77],
        [30.04],
        [31.44],
        [31.15],
        [30.58],
        [30.49],
        [30.44]
])

Xt = X.T

XtX = np.dot(Xt, X)

C = XtX.I 


XtY = np.dot(Xt, Y)

b = np.dot(C, XtY)
b = [f"{i.item():.4f}" for i in b]
print(f"Coeficientes: {b}")

Coeficientes: ['30.8850', '0.0944', '-0.0775']
